In [1]:
from tkinter import *
import numpy
import random
import time
from threading import Thread


In [2]:
window = Tk()
window_width = 900
window_height = 750
board_size = 8
space_width = window_width / board_size
space_height = window_height / board_size

window.wm_geometry(f"{window_width}x{window_height}")
window.title("Chess Board")
window.resizable(False, False)

canvas = Canvas(window, width=window_width, height=window_height)
canvas.pack()

current_best = 0
perfect_board = None
final_boards = None
max_children = 8

In [3]:
def get_random_board(size):
    return [random.randint(0, size - 1) for _ in range(board_size)]


class Board:
    def __init__(self, size=5, square_size_x=50, square_size_y=50):
        self.size = size
        self.square_size_x = square_size_x
        self.square_size_y = square_size_y
        self.color1 = "#DDDDDD"
        self.color2 = "#000000"
        self.letters = [[None for _ in range(self.size)] for _ in range(self.size)]
        self.board = [[None for _ in range(self.size)] for _ in range(self.size)]
        self.create_board()

    def set_board(self, states):
        for i in range(self.size):
            for j in range(self.size):
                if self.letters[j][i] != None:
                    canvas.delete(self.letters[j][i])
        self.letters = [[None for _ in range(self.size)] for _ in range(self.size)]
        for i in range(len(states)):
            self.set_letter(states[i], i, "♕")

    def create_board(self):
        canvas.delete("all")
        for row in range(self.size):
            for col in range(self.size):
                x1 = col * self.square_size_x
                y1 = row * self.square_size_y
                x2 = x1 + self.square_size_x
                y2 = y1 + self.square_size_y
                color = self.color1 if (row + col) % 2 == 0 else self.color2
                cell = canvas.create_rectangle(x1, y1, x2, y2, fill=color, outline="")
                self.board[row][col] = cell;

    def set_letter(self, row, col, letter):
        if self.letters[row][col] != None:
            canvas.delete(self.letters[row][col])
            del self.letters[row][col]
        x1 = col * self.square_size_x
        y1 = row * self.square_size_y
        x2 = x1 + self.square_size_x
        y2 = y1 + self.square_size_y
        x_center = (x1 + x2) / 2
        y_center = (y1 + y2) / 2
        self.letters[row][col] = canvas.create_text(x_center, y_center, font=("Courier", int(44 * space_height / 100)), text=letter,
                                                    fill="#000000" if (row + col) % 2 == 0 else "#FFFFFF")

    def randomise_board(self):
        random_board = get_random_board(self.size)
        self.set_board(random_board)


In [4]:
def fitness_function(positions):
    conflicts = 0
    n = len(positions)
    for i in range(n):
        for j in range(i + 1, n):
            if positions[i] == positions[j] or \
                    positions[i] + i == positions[j] + j or \
                    positions[i] - i == positions[j] - j:
                conflicts += 1

    fitness_score = 1.0 / (1.0 + conflicts)
    return fitness_score


board = Board(size=board_size, square_size_x=space_width, square_size_y=space_height)



def evaluate_boards(boards):
    return [fitness_function(i) for i in boards]


def calculate_weighted_percentages(numbers):
    total = sum(numbers)
    percentages = [round((number / total) * 100, 2) for number in numbers]
    return percentages


def choose_weighted_random_index(numbers):
    total = sum(numbers)
    weighted_numbers = [(number / total) for number in numbers]
    index = random.choices(range(len(numbers)), weights=weighted_numbers)[0]
    return index


def mutate_board(_board):
    size = len(_board)
    for i in _board:
        if random.random() < 0.003:
            _board[i] = random.randint(0, size - 1)

def cut(b1,b2,ratio):
    return [b1[i] if i > ratio else b2[i] for i in range(board_size)]


def cutter(_boards):
    children = []
    for i in range(0,len(_boards),2):
        children.append(cut(_boards[i],_boards[i+1],board_size/2))
        children.append(cut(_boards[i+1], _boards[i], board_size/2))
    return children

def best_board(_boards):
    best = 0
    for i in range(len(_boards)):
        if _boards[best] < _boards[i]:
            best = i
    return best

def get_perfect():
    global current_best
    global perfect_board
    global final_boards
    all_boards = [get_random_board(board_size) for _ in range(max_children)]
    evaluated = [0]
    best = 0
    survivors = []

    while evaluated[best] != 1:
        del survivors
        del evaluated
        evaluated = evaluate_boards(all_boards)
        best = best_board(evaluated)
        survivors = [all_boards[choose_weighted_random_index(evaluated)] for _ in range(max_children)]
        for i in survivors:
            mutate_board(i)
        all_boards = survivors[:]
        all_boards = cutter(all_boards)
        current_best = evaluated[best]
        #board.set_board(all_boards[best-1])

    print("Done",best)
    final_boards = all_boards
    perfect_board = all_boards[best]

display = 0

def update():
    if current_best != 1:
        print(current_best)
    else:
        board.set_board(perfect_board)
    window.after(1000,update)

def update_b(event):
    update()



In [5]:
window.bind_all("<space>", update_b)
update()
window_thread = Thread(target=get_perfect, args=())
window_thread.start()
window.mainloop()

0
Done 5
